<a href="https://colab.research.google.com/github/neerajtiwari360/understand_LLM/blob/main/microsoft/Phi-3.5-mini-instruct-onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-onnx --include cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/* --local-dir .

Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]Downloading 'cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx' to '.cache/huggingface/download/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx.385cd1b908a0d2f8634e86d30236f6dbb7ae660eb3943fd1ef5bdc3847326480.incomplete'

(…)nt4-rtn-block-32-acc-level-4/config.json: 100% 919/919 [00:00<00:00, 4.38MB/s]

(…)ock-32-acc-level-4/configuration_phi3.py: 100% 10.4k/10.4k [00:00<00:00, 32.3MB/s]

(…)n-block-32-acc-level-4/added_tokens.json: 100% 293/293 [00:00<00:00, 1.69MB/s]
Download complete. Moving file to cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/config.json
Download complete. Moving file to cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/configuration_phi3.py
Download complete. Moving file to cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/added_tokens.json

(…)n-block-32-acc-level-4/genai_config.json: 100% 1.58k/1

In [2]:
!pip install numpy
!pip install --pre onnxruntime-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


## Code Test

In [14]:
!python --version

Python 3.10.12


In [6]:
import onnxruntime_genai as og

model = og.Model('cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

# Set the max length to something sensible by default,
# since otherwise it will be set to the entire context length
search_options = {}
search_options['max_length'] = 2048

chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

text = input("Input: ")
if not text:
   print("Error, input cannot be empty")
   exit

prompt = f'{chat_template.format(input=text)}'

input_tokens = tokenizer.encode(prompt)

params = og.GeneratorParams(model)
params.set_search_options(**search_options)
params.input_ids = input_tokens
generator = og.Generator(model, params)

print("Output: ", end='', flush=True)

try:
   while not generator.is_done():
     generator.compute_logits()
     generator.generate_next_token()

     new_token = generator.get_next_tokens()[0]
     print(tokenizer_stream.decode(new_token), end='', flush=True)
except KeyboardInterrupt:
    print("  --control+c pressed, aborting generation--")

print()
del generator

Input: hi
Output:  Hello! How can I assist you today? Whether you need information, advice, or just a friendly chat, I'm here to help. What's on your mind?


## Gradio Test

In [7]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [10]:
import gradio as gr
import onnxruntime_genai as og

# Load the model and tokenizer
model = og.Model('cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

# Set search options
search_options = {}
search_options['max_length'] = 2048

chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

# Function to handle the user input and generate a response
def chatbot_response(text):
    if not text:
        return "Error, input cannot be empty"

    prompt = f'{chat_template.format(input=text)}'
    input_tokens = tokenizer.encode(prompt)

    params = og.GeneratorParams(model)
    params.set_search_options(**search_options)
    params.input_ids = input_tokens
    generator = og.Generator(model, params)

    response = ""
    try:
        while not generator.is_done():
            generator.compute_logits()
            generator.generate_next_token()

            new_token = generator.get_next_tokens()[0]
            response += tokenizer_stream.decode(new_token)
    except KeyboardInterrupt:
        response += "  --control+c pressed, aborting generation--"

    return response

# Define the Gradio interface
iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(label="Input", placeholder="Ask me something...", lines=2, interactive=True),
    outputs=gr.Textbox(label="Response", lines=5),
    live=False,  # Disable live updates (user must click submit)
    title="ONNX Runtime Chatbot",
    description="This is an AI chatbot built using ONNX Runtime with Gradio. Type your message and press Submit to get a response.",
    allow_flagging="never"  # Optionally disable flagging
)

# Launch the app with a public shareable link
iface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bbecaf728e7c15c45a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
import gradio as gr
import onnxruntime_genai as og

# Load the model and tokenizer
model = og.Model('cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

# Set search options
search_options = {}
search_options['max_length'] = 2048

# Initialize the message history
messages = [
    {"role": "system", "content": "You are a pirate chatbot name wishper, who always responds in pirate speak!"},
    {"role": "user", "content": "your name is Jon"}
]

chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

# Function to handle the user input and generate a response
def chatbot_response(text):
    if not text:
        return "Error, input cannot be empty"

    # Append the user's message to the message history
    messages.append({"role": "user", "content": text})

    # Format the conversation for the prompt
    prompt = f'{chat_template.format(input=text)}'
    input_tokens = tokenizer.encode(prompt)

    params = og.GeneratorParams(model)
    params.set_search_options(**search_options)
    params.input_ids = input_tokens
    generator = og.Generator(model, params)

    response = ""
    try:
        while not generator.is_done():
            generator.compute_logits()
            generator.generate_next_token()

            new_token = generator.get_next_tokens()[0]
            response += tokenizer_stream.decode(new_token)
    except KeyboardInterrupt:
        response += "  --control+c pressed, aborting generation--"

    # Add the bot's response to the message history
    messages.append({"role": "assistant", "content": response.strip()})

    return response.strip()

# Define the Gradio interface
iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(label="Input", placeholder="Ask me something...", lines=2, interactive=True),
    outputs=gr.Textbox(label="Response", lines=5),
    live=False,  # Disable live updates (user must click submit)
    title="Pirate Chatbot",
    description="This is a pirate chatbot that responds in pirate speak. Type your message and press Submit to get a response.",
    allow_flagging="never"  # Optionally disable flagging
)

# Launch the app with a public shareable link
iface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6aa64dc2a8d09363ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
